In [1]:
import rebound
import pandas as pd
from rebound import InterruptiblePool
import numpy as np
import os
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

path = '/mnt/raid-cita/dtamayo/newstabilitydataset/data/resonant/simulation_archives/runs/'

In [2]:
def generate_features(sim):
    import pandas as pd
    import numpy as np
    import rebound
    from collections import OrderedDict

    maxorbs = 1e4    # how many orbits to do
    Nout = 100        # how many output samples
    window = 10       # number of points to take for the 'window' features
    
    ps = sim.particles
    P0 = ps[1].P
    tmax = maxorbs * P0 # number of inner planet orbital periods to integrate
    
    sim.collision_resolve = collision

    times = np.linspace(0,tmax,Nout)
    
    a = np.zeros((sim.N,Nout))
    e = np.zeros((sim.N,Nout))
    inc = np.zeros((sim.N,Nout))
    
    beta12 = np.zeros( Nout)
    beta23 = np.zeros(Nout)

    Rhill12 = ps[1].a*((ps[1].m+ps[2].m)/3.)**(1./3.)
    Rhill23 = ps[2].a*((ps[2].m+ps[3].m)/3.)**(1./3.)
    
    eHill = [0, Rhill12/ps[1].a, max(Rhill12, Rhill23)/ps[2].a, Rhill23/ps[3].a]
    daOvera = [0, (ps[2].a-ps[1].a)/ps[1].a, min(ps[3].a-ps[2].a, ps[2].a-ps[1].a)/ps[2].a, (ps[3].a-ps[2].a)/ps[3].a]
    
    for i, t in enumerate(times):
        for j in [1,2,3]:
            a[j,i] = ps[j].a
            e[j,i] = ps[j].e
            inc[j,i] = ps[j].inc

        #need to update rhills?
        Rhill12 = ps[1].a*((ps[1].m+ps[2].m)/3.)**(1./3.)
        Rhill23 = ps[2].a*((ps[2].m+ps[3].m)/3.)**(1./3.)
        
        beta12[i] = (ps[2].a - ps[1].a)/Rhill12
        beta23[i] = (ps[3].a - ps[2].a)/Rhill23   
        sim.integrate(t)
    
    features = OrderedDict()
    
    for string, feature in [("beta12", beta12), ("beta23", beta23)]:
        mean = feature.mean()
        std = feature.std()
        features["avg_"+string] = mean
        features["std_"+string] = std
        features["min_"+string] = min(feature)
        features["max_"+string] = max(feature)

    for j in [1,2,3]:
        for string, feature in [('a', a), ('e', e), ('inc', inc)]:
            mean = feature[j].mean()
            std = feature[j].std()
            features['avg_'+string+str(j)] = mean
            features['std_'+string+str(j)] = std
            features['max_'+string+str(j)] = feature[j].max()
            features['min_'+string+str(j)] = feature[j].min()
            features['norm_std_'+string+str(j)] = std/mean
            features['norm_max_'+string+str(j)] = np.abs(feature[j] - mean).max()/mean
            sample = feature[j][:window]
            samplemean = sample.mean()
            features['norm_std_window'+str(window)+'_'+string+str(j)] = sample.std()/samplemean
            features['norm_max_window'+str(window)+'_'+string+str(j)] = np.abs(sample - samplemean).max()/samplemean

        for string, feature in [('eH', e), ('iH', inc)]:
            mean = feature[j].mean()
            std = feature[j].std()

            features['avg_'+string+str(j)] = mean/eHill[j]
            features['std_'+string+str(j)] = std/eHill[j]
            features['max_'+string+str(j)] = feature[j].max()/eHill[j]
            features['min_'+string+str(j)] = feature[j].min()/eHill[j]

        string, feature = ('ecross', e)
        features['avg_'+string+str(j)] = mean/daOvera[j]
        features['std_'+string+str(j)] = std/daOvera[j]
        features['max_'+string+str(j)] = feature[j].max()/daOvera[j]
        features['min_'+string+str(j)] = feature[j].min()/daOvera[j]

    return pd.DataFrame([pd.Series(features, index=list(features.keys()))])

/mnt/raid-cita/dtamayo/oldfeaturesenv/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
def chunkify(lst,n):
    return [lst[i::n] for i in range(n)]

In [4]:
def worker(df):
    Ndump=10
    for i in range(df.shape[0]):
        #try:
        run(df, i, tmax=1.e4)
        #except:
        #    for col in columns:
        #        df.loc[df.index[i], col] = np.nan
        if i%Ndump == 0:
            df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')
    df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')

In [5]:
features = []
features += ['avg_iH1', 'avg_iH2']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']
features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']
#features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ['avg_beta12','std_beta12','min_beta12','max_beta12','avg_beta23','std_beta23','min_beta23','max_beta23'] 

In [6]:
featuresfile='resoldfeatures.csv'

In [36]:
df = pd.DataFrame(index=range(10000), columns=features)
df.tail()

,avg_iH1,avg_iH2,norm_std_a1,norm_max_a1,norm_std_window10_a1,norm_max_window10_a1,norm_std_a2,norm_max_a2,norm_std_window10_a2,norm_max_window10_a2,...,max_ecross3,min_ecross3,avg_beta12,std_beta12,min_beta12,max_beta12,avg_beta23,std_beta23,min_beta23,max_beta23
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
def writedata(row):
    file = path+"sa{0:07d}.bin".format(row.name)
    sim = rebound.Simulation.from_file(file)
    return feature_function(sim)[features].loc[0]

In [40]:
%%time
df = df.apply(writedata, axis=1)
df.tail()

Escape: ('User caused exit. Simulation did not finish.', u'occurred at index 45')

In [37]:
df.to_csv(featuresfile, encoding='ascii')

In [9]:
Nworkers=60
df = pd.read_csv(featuresfile, index_col=0)
df.tail()

,runstring,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,maxi1,maxi2,maxstrength,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
24995,0024995.bin,1.153346,0.171907,0.387978,0.184376,9.0,2.0,2.0,3.0,0.003431,...,1.745551e-06,0.000717,0.060754,0.005449,0.016285,0.002086,0.084769,0.001440,0.367484,0.084805
24996,0024996.bin,2.964576,0.118284,0.220488,0.091394,8.0,1.0,2.0,3.0,0.006414,...,4.867103e-07,0.002863,1.386549,0.311781,6.123500,2.000274,0.752978,0.014807,0.123352,-0.057720
24997,0024997.bin,2.349224,0.579004,0.697671,0.075153,2.0,1.0,1.0,2.0,0.019195,...,-1.135500e-07,0.001531,0.053027,0.011746,0.574987,0.021306,1.199827,0.018510,0.096530,-0.112411
24998,0024998.bin,2.939841,0.055424,0.286817,0.219241,4.0,1.0,2.0,3.0,0.003782,...,1.357478e-08,0.000079,0.125268,0.004959,0.336359,0.010575,9.060472,0.015823,0.091789,0.004391
24999,0024999.bin,2.884900,0.316695,0.849302,0.404503,3.0,1.0,1.0,2.0,0.002190,...,1.085541e-11,0.000023,0.210186,0.039263,0.573982,0.045580,0.208377,0.003629,0.008251,0.000276


In [10]:
df = df[df['featgentime'] == 0.]
print(df.shape)
IDs = df.index.values
splits = chunkify(IDs, Nworkers)
dfs = [df.loc[split] for split in splits]

(707, 56)


In [11]:
pool = InterruptiblePool(Nworkers)
pool.map(worker, dfs)

/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary fil

/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might h

/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct

/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:31: RuntimeWarning: invalid value encountered in sqrt
  Xstarres = -0.5*np.sqrt(3.*Phiprime+2.)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:112: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:114: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed

...

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:102: RankWarning: Polyfit may be poorly conditioned
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:31: RuntimeWarning: invalid value encountered in sqrt
  Xstarres = -0.5*np.sqrt(3.*Phiprime+2.)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:102: RankWarning: Polyfit may be poorly conditioned
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:122: RankWarning: Polyfit may be poorly conditioned
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:102: RankWarning: Polyfit may be poorly conditioned
/mnt/raid-cita/dtama

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:102: RankWarning: Polyfit may be poorly conditioned
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:122: RankWarning: Polyfit may be poorly conditioned
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:31: RuntimeWarning: invalid value encountered in sqrt
  Xstarres = -0.5*np.sqrt(3.*Phiprime+2.)
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:270: RuntimeWarning: invalid value encountered in double_scalars
  andvars.dKprime = ((j-k)*p['m2']*p['sLambda20']*dL2hat + j*p['m1']*p['sLambda10']*dL1hat)/p['K0']
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:29: Runtime

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:29: RuntimeWarning: invalid value encountered in double_scalars
  Xstarres = np.sqrt(-Phiprime)*np.sinh(1./3.*np.arcsinh(-abs(Phiprime)**(-1.5)))
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:29: RuntimeWarning: invalid value encountered in double_scalars
  Xstarres = np.sqrt(-Phiprime)*np.sinh(1./3.*np.arcsinh(-abs(Phiprime)**(-1.5)))
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:29: RuntimeWarning: invalid value encountered in double_scalars
  Xstarres = np.sqrt(-Phiprime)*np.sinh(1./3.*np.arcsinh(-abs(Phiprime)**(-1.5)))
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:102: RankWarni

/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:270: RuntimeWarning: invalid value encountered in double_scalars
  andvars.dKprime = ((j-k)*p['m2']*p['sLambda20']*dL2hat + j*p['m1']*p['sLambda10']*dL1hat)/p['K0']
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:102: RankWarning: Polyfit may be poorly conditioned
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:112: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:114: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:114: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:125: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a fut

In [71]:
df = pd.read_csv('/mnt/raid-cita/dtamayo/newstabilitydataset/data/csvs/data.csv', index_col=0)
df['SimID'] = np.arange(df.shape[0])
df['Stable'] = df['instability_time'] >= 9.99e8
df = df[['SimID', 'Stable', 'instability_time', 'runstring', 'seed']]
for col in columns:
    df[col] = 0
df.set_index('SimID', inplace=True)
df = df.iloc[:10]
df.head()

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
0,False,1.545872e+06,0000000.bin,3500000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,True,9.990000e+08,0000001.bin,700001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,True,9.990000e+08,0000002.bin,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,False,2.287671e+06,0000003.bin,2100003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,False,9.668931e+05,0000004.bin,700004,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
import os

df = pd.read_csv(featuresfile, index_col=0)

for f in os.listdir('tempcsvs/'):
    if f[-4:] == '.csv':
        print(f)
        newdf = pd.read_csv('tempcsvs/'+f, index_col=0)
        df.update(newdf)
        
#test = test.sort_values('SimID')
#test = test.reset_index(drop=True)
df[df['featgentime'] == 0.].shape

102952.csv
102975.csv
106886.csv
106893.csv
106890.csv
102992.csv
102967.csv
102966.csv
106899.csv
102991.csv
102957.csv
102996.csv
102976.csv
102956.csv
102972.csv
102951.csv
102958.csv
106906.csv
102989.csv
102993.csv
106921.csv
102955.csv
106881.csv
106880.csv
102949.csv
106900.csv
106889.csv
102980.csv
102981.csv
102995.csv
102985.csv
106894.csv
102982.csv
106884.csv
102986.csv
106897.csv
106922.csv
106905.csv
106898.csv
106914.csv
102974.csv
102990.csv
102963.csv
102977.csv
106909.csv
102968.csv
102965.csv
106923.csv
102969.csv
106912.csv
106915.csv
106908.csv
102984.csv
102983.csv
102988.csv
106895.csv
102948.csv
102962.csv
106896.csv
106916.csv
102959.csv
102973.csv
106902.csv
106885.csv
102987.csv
106883.csv
106903.csv
106919.csv
102978.csv
106892.csv
102970.csv
102979.csv
102953.csv
106879.csv
102960.csv
106891.csv
106913.csv
102964.csv
106918.csv
106911.csv
102950.csv
106901.csv
106904.csv
106907.csv
106920.csv
106910.csv
106882.csv


In [16]:
df.to_csv(featuresfile, encoding='ascii')

In [68]:
df[df['featgentime'] == 0]

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
2058,False,NaN,0002058.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
test = pd.read_csv('test.csv', index_col=0)
test.tail()

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
9995,True,9.990000e+08,0009995.bin,1409995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,False,3.814426e+06,0009996.bin,2809996,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,False,1.498363e+04,0009997.bin,1409997,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,False,9.419807e+05,0009998.bin,709998,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9999,False,1.895071e+07,0009999.bin,9999,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [191]:
import os

#test = pd.DataFrame(columns=test.columns)
for f in os.listdir('tempcsvs/'):
    if f[-4:] == '.csv':
        print(f)
        newdf = pd.read_csv('tempcsvs/'+f, index_col=0)
        test.update(newdf)
        
#test = test.sort_values('SimID')
#test = test.reset_index(drop=True)
df[df['featgentime'] == 0.]



24103.csv
23373.csv
24110.csv
24098.csv
23372.csv
24350.csv
24339.csv
24343.csv
24337.csv
24108.csv
23375.csv
23138.csv
23371.csv
24340.csv
24109.csv
24112.csv
24349.csv
24097.csv
24107.csv
24111.csv
24101.csv
23141.csv
23148.csv
23136.csv
23384.csv
24338.csv
24344.csv
24336.csv
24345.csv
23377.csv
24351.csv
24106.csv
23150.csv
24341.csv
23381.csv
23376.csv
24346.csv
23380.csv
24342.csv
23379.csv
24348.csv
24105.csv
23135.csv
24099.csv
23144.csv
23385.csv
23134.csv
23370.csv
24104.csv
24102.csv
23146.csv
23383.csv
24347.csv
23140.csv
23142.csv


,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
0,False,1.545872e+06,0000000.bin,3500000.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
1,True,9.990000e+08,0000001.bin,700001.0,14.756695,0.240504,0.340424,0.080547,9.0,1.0,...,0.779525,0.797556,0.008385,0.005011,1.420017e-09,0.000092,0.0,0.0,0.0,0.023081
2,True,9.990000e+08,0000002.bin,2.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
3,False,2.287671e+06,0000003.bin,2100003.0,3.098584,0.406112,1.007603,0.427768,5.0,2.0,...,0.115293,0.150077,0.004127,0.001260,5.512634e-08,0.000386,0.0,0.0,0.0,0.039200
4,False,9.668931e+05,0000004.bin,700004.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
5,False,5.755894e+05,0000005.bin,1400005.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
6,False,7.248372e+07,0000006.bin,6.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
7,False,2.225858e+04,0000007.bin,2100007.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
8,False,2.930966e+05,0000008.bin,8.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000


In [100]:
test[test['featgentime']==0].shape

In [99]:
test.to_csv('newshortfeatures.csv')

In [61]:
df.to_csv('shortfeatures.csv', encoding='ascii')

In [171]:
test.to_csv('test.csv', encoding='ascii')

In [186]:
test = pd.read_csv('test.csv', index_col=0)
test.tail()

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
9995,True,9.990000e+08,0009995.bin,1409995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,False,3.814426e+06,0009996.bin,2809996,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,False,1.498363e+04,0009997.bin,1409997,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,False,9.419807e+05,0009998.bin,709998,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9999,False,1.895071e+07,0009999.bin,9999,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
test.shape

In [193]:
test = pd.read_csv('test.csv', index_col=0)
test = test.iloc[:100]
test.tail()

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
95,False,3.023697e+04,0000095.bin,95,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,False,1.661314e+07,0000096.bin,96,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,False,1.017439e+07,0000097.bin,97,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,True,1.000000e+09,0000098.bin,1400098,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,False,4.521761e+04,0000099.bin,99,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
test.head()

,SimID,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
0,0,False,1.545872e+06,0000000.bin,3500000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,True,9.990000e+08,0000001.bin,700001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,True,9.990000e+08,0000002.bin,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,False,2.287671e+06,0000003.bin,2100003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,False,9.668931e+05,0000004.bin,700004,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
test['SimID'].values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [44]:
def worker(df):
    Ndump=10
    for i, row in df.iterrows():
        run(df, row['SimID'])
        if i%Ndump == 0:
            df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')
    df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')

In [48]:
def run(df, ID):
    if ID % 2 == 0:
        df.loc[df['SimID']==ID, 'featgentime'] = 7

In [300]:
test.head()

,SimID,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
0,0,False,1.545872e+06,0000000.bin,3500000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,True,9.990000e+08,0000001.bin,700001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,True,9.990000e+08,0000002.bin,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,False,2.287671e+06,0000003.bin,2100003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,False,9.668931e+05,0000004.bin,700004,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
Nworkers=3
test = test[test['featgentime'] == 0]
print(test.shape)
IDs = test['SimID'].values
splits = chunkify(IDs, Nworkers)
dfs = [test[test['SimID'].isin(split)] for split in splits]
pool = InterruptiblePool(Nworkers)
pool.map(worker, dfs)

(33, 35)


[None, None, None]

In [84]:
import os

test = pd.DataFrame(columns=test.columns)
for f in os.listdir('tempcsvs/'):
    if f[-4:] == '.csv':
        print(f)
        newdf = pd.read_csv('tempcsvs/'+f, index_col=0)
        test = test.append(newdf)
        
test = test.sort_values('SimID')
test = test.reset_index(drop=True)
test.head()

10377.csv
10381.csv
10371.csv
10375.csv
10385.csv
10376.csv
10384.csv
10386.csv
10379.csv
10383.csv
10382.csv
10378.csv
10373.csv


,DiffcoeffZ12,DiffcoeffZ23,SimID,Stable,Zcosphistd12,Zcosphistd23,Zcross12,Zcross13,Zcross23,Zdetrendedstd12,...,maxi2,maxj,maxk,maxstrength,medvarZ12,medvarZ23,runstring,secondres,secondstrength,seed
0,0.000000e+00,-5.249295e-10,0.0,False,0.000000,0.012035,0.504063,0.594658,0.060234,0.000000,...,3.0,12.0,1.0,0.048244,0.000000,1.471706e-04,0000000.bin,0.195509,0.0,3500000.0
1,5.530677e-11,9.176673e-11,2.0,True,0.003070,0.000785,0.129660,1.261558,1.001981,0.003071,...,2.0,6.0,1.0,0.067762,0.000009,3.508728e-07,0000002.bin,0.029946,0.0,2.0
2,-1.838390e-08,-5.059799e-09,4.0,False,0.023602,0.033524,0.059897,0.332922,0.257596,0.022934,...,2.0,12.0,1.0,0.030171,0.000586,1.147590e-03,0000004.bin,0.140455,0.0,700004.0
3,2.151018e-08,-1.581556e-07,5.0,False,0.011011,0.053589,0.185250,0.259467,0.062617,0.001381,...,3.0,23.0,2.0,0.001197,0.000029,2.971640e-03,0000005.bin,0.567226,0.0,1400005.0
4,0.000000e+00,-5.992155e-10,6.0,False,0.000000,0.009534,0.516020,0.760215,0.161076,0.000000,...,3.0,5.0,1.0,0.111359,0.000000,8.956325e-05,0000006.bin,-1.000000,0.0,6.0


In [88]:
test.head(50)

,DiffcoeffZ12,DiffcoeffZ23,SimID,Stable,Zcosphistd12,Zcosphistd23,Zcross12,Zcross13,Zcross23,Zdetrendedstd12,...,maxi2,maxj,maxk,maxstrength,medvarZ12,medvarZ23,runstring,secondres,secondstrength,seed
0,0.000000e+00,-5.249295e-10,0.0,False,0.000000,0.012035,0.504063,0.594658,0.060234,0.000000,...,3.0,12.0,1.0,0.048244,0.000000,1.471706e-04,0000000.bin,0.195509,0.0,3500000.0
1,5.530677e-11,9.176673e-11,2.0,True,0.003070,0.000785,0.129660,1.261558,1.001981,0.003071,...,2.0,6.0,1.0,0.067762,0.000009,3.508728e-07,0000002.bin,0.029946,0.0,2.0
2,-1.838390e-08,-5.059799e-09,4.0,False,0.023602,0.033524,0.059897,0.332922,0.257596,0.022934,...,2.0,12.0,1.0,0.030171,0.000586,1.147590e-03,0000004.bin,0.140455,0.0,700004.0
3,2.151018e-08,-1.581556e-07,5.0,False,0.011011,0.053589,0.185250,0.259467,0.062617,0.001381,...,3.0,23.0,2.0,0.001197,0.000029,2.971640e-03,0000005.bin,0.567226,0.0,1400005.0
4,0.000000e+00,-5.992155e-10,6.0,False,0.000000,0.009534,0.516020,0.760215,0.161076,0.000000,...,3.0,5.0,1.0,0.111359,0.000000,8.956325e-05,0000006.bin,-1.000000,0.0,6.0
5,5.718216e-06,-3.337570e-07,7.0,False,0.438043,0.177081,0.069323,0.215457,0.136660,0.058753,...,3.0,4.0,1.0,0.003488,0.032111,8.217204e-03,0000007.bin,0.393146,0.0,2100007.0
6,0.000000e+00,-4.990015e-10,8.0,False,0.000000,0.024843,0.492813,0.657701,0.110455,0.000000,...,3.0,7.0,1.0,0.037960,0.000000,6.195017e-04,0000008.bin,-1.000000,0.0,8.0
7,7.087217e-09,-2.932458e-08,10.0,False,0.018350,0.033251,0.130162,0.381586,0.222467,0.013577,...,2.0,6.0,1.0,0.023995,0.000261,1.032598e-03,0000010.bin,0.205764,0.0,700010.0
8,1.371793e-08,3.786168e-09,11.0,False,0.079462,0.084492,0.093278,0.324455,0.211454,0.007644,...,3.0,4.0,1.0,0.556784,0.000062,2.711810e-04,0000011.bin,0.083947,0.0,2100011.0
9,9.646483e-10,4.714798e-09,12.0,True,0.028829,0.015754,0.136068,0.340661,0.180089,0.007188,...,2.0,6.0,1.0,0.001430,0.000051,9.376263e-05,0000012.bin,0.553938,0.0,12.0


In [86]:
test.shape